Ce Notebook est une brève introduction aux échecs informatiques modernes. Les moteurs d'échecs utilisant l'intelligence artificielle basée sur l'apprentissage profond (deep learning) ont eu un impact considérable sur la communauté des joueurs d'échecs. AlphaZero, le monstre d'échecs développé par DeepMind, la filiale de recherche de Google, a soudain semblé capable de jouer de belles parties d'échecs, presque semblables à celles d'un humain. Certaines parties montraient des sacrifices de pièces sans gain tactique immédiat, mais des avantages positionnels à long terme que les moteurs d'échecs courants étaient incapables de trouver. Nous commencerons par examiner les méthodes de recherche utilisées par les moteurs d'échecs. Nous apprenons comment les moteurs d'échecs "classiques" utilisent la recherche alpha-bêta, et comment la recherche par arbre de Monte-Carlo offre une alternative pour les échecs et d'autres jeux plus difficiles tels que le Shogi ou le Go. À titre d'exemple, nous mettrons en œuvre la recherche alpha-bêta et la recherche arborescente de Monte-Carlo et nous les testerons avec une position d'échecs simple.